# More tuning

In [1]:
import nbloader
from demo_process import *
print ("Packages loaded.")

Packages loaded.


#### Preprocess data

In [2]:
dirPath = '../data'
id2idx,nTotalTxt,nTotalChildren = get_unique_idx(_dirPath=dirPath,_VERBOSE=False)
totalFeats,trajList,uniqueChildIdxList,titleList \
    = preprocess_traj(_dirPath=dirPath,_nTotalTxt=nTotalTxt
                    ,_nTotalChildren=nTotalChildren,_id2idx=id2idx
                    ,_VERBOSE=False,_PLOT_SMOOTHED_TRAJ=False)
print ("Preprocess done.")

Preprocess done.


#### Animate (takes time)

In [3]:
ANIMATE_TRAJ = False
if ANIMATE_TRAJ:
    fIdx,nTick = 0,100
    frames = make_frames(_traj=trajList[fIdx]
                         ,_uniqueChildList=uniqueChildIdxList[fIdx],_nTick=nTick)
    animate_frames(_frames=frames,_figsize=(8,8),_title=titleList[fIdx])

#### Test classification results

In [35]:
nCluster,chunkSec,reduceDim  = 5,10,10 # 5,10,5:60% / 
nTrain = 40 # Use 40 children for traning, 18 for testing
surveyRes = get_surveyRes()
nzrFeat,nzdTotalFeats = get_noramlized_feat(totalFeats,_VERBOSE=False)
chunks_total = split_trajs(nzdTotalFeats,chunkSec,_VERBOSE=False)
nTest = len(chunks_total) - nTrain
centers,Ured = reduce_dim_n_cluster(chunks_total,reduceDim,nCluster
                                    ,nTrain,nTest,_VERBOSE=False,_PLOT_HIST=False)
featHistTotal = get_featHist(chunks_total,centers,Ured,nCluster,_VERBOSE=False)
xTrain,yTrain = featHistTotal[:nTrain,:],surveyRes[:nTrain,:]
xTest,yTest = featHistTotal[nTrain:,:],surveyRes[nTrain:,:]
avgAccr,accuracy_list = run_xgboost(xTrain,yTrain,xTest,yTest,surveyRes.shape[1]
                ,_VERBOSE=False,_eta=0.1,_gamma=10,_max_depth=5,_num_round=50
                ,_min_child_weight=3,_colsample_bytree=0.7)
np.set_printoptions(precision=3)
print ("Avg accr:%.2f%% (%s)"%(avgAccr*100,accuracy_list))

Avg accr:60.32% ([0.611 0.5   0.389 0.5   0.722 0.722 0.778])
